In [1]:
!pip install selenium

In [2]:
import selenium.webdriver   # модуль необходим для автоматизации взаимодействия с веб-браузерами

In [1]:
pip install --upgrade undetected-chromedriver

  Using cached undetected_chromedriver-3.5.5-py3-none-any.whl
  Attempting uninstall: undetected-chromedriver
    Found existing installation: undetected-chromedriver 3.5.3
    Uninstalling undetected-chromedriver-3.5.3:
      Successfully uninstalled undetected-chromedriver-3.5.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import os
import time
import glob

# Настройка параметров загрузки
download_dir = r"C:\Users\Катерина Андреевна\code\парсинг\pdf + supplementary information"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
    print(f"Создана папка: {download_dir}")

# Конфигурация ChromeOptions
options = uc.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True,
    "profile.default_content_settings.popups": 0
}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Инициализация драйвера
driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 30)

def wait_for_download(download_dir, timeout=60):
    print("Ожидание загрузки файла...")
    start_time = time.time()
    while time.time() - start_time < timeout:
        files = glob.glob(os.path.join(download_dir, "*.pdf"))
        if files:
            print(f"Файл загружен: {files[0]}")
            return True
        time.sleep(1)
    return False

try:
    print("Открываем страницу статьи")
    driver.get('https://doi.org/10.2903/j.efsa.2019.5681')
    
    # Даем странице полностью загрузиться
    time.sleep(5)
    
    # Шаг 1: Клик по кнопке ePDF (используем несколько вариантов локаторов)
    try:
        # Вариант 1: По классу pdf-download
        epdf_button = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.pdf-download'))
        )
    except:
        try:
            # Вариант 2: По атрибуту title
            epdf_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[title="ePDF"]'))
            )
        except:
            # Вариант 3: По XPath с учетом структуры
            epdf_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "PdfLink")]//a[contains(@class, "pdf-download")]'))
            )
    
    print("Кнопка ePDF найдена, скроллим к элементу")
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", epdf_button)
    time.sleep(1)
    
    print("Нажатие кнопки ePDF")
    # Пробуем разные способы клика
    try:
        epdf_button.click()
    except:
        driver.execute_script("arguments[0].click();", epdf_button)
    print("Кнопка ePDF нажата")
    
    # Шаг 2: Проверка открытия новой вкладки
    print(f"Количество открытых вкладок: {len(driver.window_handles)}")
    time.sleep(3)  # Даем время для открытия новой вкладки
    
    if len(driver.window_handles) > 1:
        driver.switch_to.window(driver.window_handles[1])
        print("Переключились на новую вкладку")
    else:
        print("Новая вкладка не открылась, работаем в текущей")
        # Если новая вкладка не открылась, возможно PDF начал скачиваться сразу
        if wait_for_download(download_dir):
            print("Файл PDF успешно загружен")
            driver.quit()
            exit()
    
    # Шаг 3: Ожидание кнопки скачивания в новой вкладке
    try:
        download_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, '//a[contains(@class, "download") and contains(@aria-label, "PDF")]'))
        )
    except:
        try:
            download_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[title*="Download PDF"]'))
            )
        except:
            # Если кнопка не найдена, проверяем, не началась ли загрузка автоматически
            if wait_for_download(download_dir):
                print("Файл PDF успешно загружен")
                driver.quit()
                exit()
            else:
                raise Exception("Не удалось найти кнопку скачивания PDF")
    
    print("Найдена кнопка скачивания")
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", download_button)
    time.sleep(1)
    
    # Шаг 4: Клик по кнопке скачивания
    print("Клик по кнопке скачивания")
    try:
        download_button.click() 
    except:
        driver.execute_script("arguments[0].click();", download_button)
    
    # Шаг 5: Ожидание загрузки файла
    if wait_for_download(download_dir):
        print("Файл успешно загружен")
    else:
        print("Файл не был загружен в течение времени ожидания")

finally:
    # Закрываем все вкладки и драйвер
    driver.quit()
    print("Браузер закрыт")

print(f"Проверьте папку загрузки: {download_dir}")

Открываем страницу статьи
Кнопка ePDF найдена, скроллим к элементу
Нажатие кнопки ePDF
Кнопка ePDF нажата
Количество открытых вкладок: 1
Новая вкладка не открылась, работаем в текущей
Ожидание загрузки файла...
Найдена кнопка скачивания
Клик по кнопке скачивания
Ожидание загрузки файла...
Файл загружен: C:\Users\Катерина Андреевна\code\парсинг\pdf + supplementary information\EFSA Journal - 2019 -  - Safety evaluation of the food enzyme alpha‐amylase from a genetically modified Bacillus subtilis .pdf
Файл успешно загружен
Браузер закрыт
Проверьте папку загрузки: C:\Users\Катерина Андреевна\code\парсинг\pdf + supplementary information
